# Decision Trees

What is Gini Impurity?
Gini Impurity is a measure of how often a randomly chosen element from a set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. It ranges from 0 to 0.5:

0: The set is perfectly pure (all elements belong to the same class).

0.5: The set is maximally impure (elements are evenly distributed across classes).



In [1]:
import numpy as np

In [2]:
class DecisionTreeNode:
    def __init__ (self, feature_index=None, threshold=None, left=None, right=None, value=None):
        """
         A node in the decision tree.
        - feature_index: Index of the feature used for splitting.
        - threshold: Threshold value for the split.
        - left: Left child node (samples <= threshold).
        - right: Right child node (samples > threshold).
        - value: Predicted value if the node is a leaf.
        """

        self.feature_index=feature_index
        self.threshold=threshold
        self.left=left 
        self.right=right
        self.value=value 

In [4]:
class DecisionTree:
    def __init__(self, max_depth=None):
        """ 
        Decision Tree Classifier.
        - max_depth: Maximum depth of the tree.        
        """
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        """ 
        Build the decision tree.
        - X: Feature matrix (n_samples, n_features).
        - y: Target vector (n_samples,).        
        """
        self.root=self._grow_tree(X, y)

    def predict(self, X):
        """ 
        Predict the target for input data.
        - X: Feature matrix (n_samples, n_features).        
        """
        return np.array([self._predict(inputs) for inputs in X])
    
    def _grow_tree(self, X, y, depth=0):
        """ 
        Recursively grow the decision tree.
        - X: Feature matrix.
        - y: Target vector.
        - depth: Current depth of the tree.        
        """
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions:
        # 1. All samples have the same label.
        # 2. Maximum depth is reached.
        # 3. No features left to split on.
        if n_labels==1 or depth==self.max_depth or n_samples < 2:
            left_value = self._most_common_label(y)
            return DecisionTreeNode(value=left_value)
        
        # Find the best split
        feature_index, threshold = self._best_split(X, y)

        # Split the data
        left_indices = X[:, feature_index] <= threshold 
        right_indices = X[:, feature_index] > threshold
        X_left, y_left = X[left_indices], y[left_indices] 
        X_right, y_right = X[right_indices], y[right_indices]

        # Recursively grow the left and right subtree
        left_child = self._grow_tree(X_left, y_left, depth + 1)
        right_child = self._grow_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature_index, threshold, left_child, right_child)
    
    def _best_split(self, X, y):
        """
        Find the best feature and threshold to split the data.
        - X: Feature matrix.
        - y: Target vector.
        """
        n_samples, n_features = X.shape
        best_gini = float('inf')
        best_feature_index, best_threshold = None, None

        # Iterate over all features and thresholds to find the best split 
        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_indices = X[:, feature_index] <= threshold
                right_indices = X[:, feature_index] > threshold 
                if sum(left_indices) == 0 or sum(right_indices) == 0:
                    continue 

                #calculate gini impurity for the split 
                gini = self._gini(y[left_indices]) * sum(left_indices)/n_samples + \
                    self._gini(y[right_indices]) * sum(right_indices)/n_samples 

                if gini < best_gini:
                    best_gini = gini
                    best_feature_index = feature_index
                    best_threshold=threshold

        return best_feature_index, best_threshold

    def _gini(self, y):
        """ 
        Calculate the gini impurity for a set of labels.
        - y : Target vector
        """      
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)
    
    def _most_common_label(self, y):
        """ 
        Return the most common label in a set.
        - y : Target vector
        """

        return np.bincount(y).argmax()
    
    def _predict(self, inputs):
        """ 
        Predicts the target for a single input by traversing the tree. 
        - inputs: A single sample's features
        """

        node = self.root 
        while node.value is None:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right 
        return node.value 

In [5]:
# Example usage
if __name__ == "__main__":
    # Sample dataset
    X = np.array([[2, 3], [10, 15], [3, 4], [6, 8], [7, 10], [8, 12]])
    y = np.array([0, 1, 0, 1, 1, 1])

    # Train the decision tree
    tree = DecisionTree(max_depth=3)
    tree.fit(X, y)

    # Make predictions
    predictions = tree.predict(X)
    print("Predictions:", predictions)

Predictions: [0 1 0 1 1 1]
